In [ ]:
###Merged Data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os


In [6]:
fear_greed = pd.read_csv("fear_greed_index.csv")
trader = pd.read_csv("historical_data.csv")


In [7]:
fear_greed['date'] = pd.to_datetime(fear_greed['date']).dt.date
trader['Timestamp'] = pd.to_datetime(trader['Timestamp'], unit='ms')
trader['date'] = trader['Timestamp'].dt.date


In [8]:
trader.columns = trader.columns.str.lower().str.replace(" ", "_")


In [9]:
daily_trader = trader.groupby('date').agg(
    total_pnl=('closed_pnl', 'sum'),
    trade_count=('trade_id', 'count')
).reset_index()


In [10]:
merged_data = pd.merge(
    daily_trader,
    fear_greed[['date', 'classification']],
    on='date',
    how='inner'
)
merged_data.head()


,date,total_pnl,trade_count,classification
0,2023-03-28,0.000000e+00,3,Greed
1,2023-11-14,1.555034e+02,1045,Greed
2,2024-03-09,1.769655e+05,6962,Extreme Greed
3,2024-07-03,1.587424e+05,7141,Neutral
4,2024-10-27,3.189461e+06,35241,Greed


In [11]:
os.makedirs("csv_files", exist_ok=True)
merged_data.to_csv("csv_files/merged_data.csv", index=False)


In [13]:
sentiment_summary = merged_data.groupby("classification").agg(
    avg_daily_pnl=("total_pnl", "mean"),
    avg_trade_count=("trade_count", "mean"),
    median_pnl=("total_pnl", "median"),
    max_pnl=("total_pnl", "max")
).reset_index()

sentiment_summary


,classification,avg_daily_pnl,avg_trade_count,median_pnl,max_pnl
0,Extreme Greed,1.769655e+05,6962.000000,1.769655e+05,1.769655e+05
1,Fear,6.699925e+06,133871.000000,6.699925e+06,6.699925e+06
2,Greed,1.063206e+06,12096.333333,1.555034e+02,3.189461e+06
3,Neutral,1.587424e+05,7141.000000,1.587424e+05,1.587424e+05


In [ ]:
#Correlation

correlation = merged_data[
    ["total_pnl", "trade_count"]
].corr()

correlation


,total_pnl,trade_count
total_pnl,1.000000,0.974254
trade_count,0.974254,1.000000


In [ ]:
# Profit vs Loss Days by Sentiment

merged_data["profit_day"] = merged_data["total_pnl"].apply(
    lambda x: "Profit" if x > 0 else "Loss"
)

profit_distribution = (
    merged_data.groupby(["classification", "profit_day"])
    .size()
    .unstack(fill_value=0)
)

profit_distribution


profit_day,Loss,Profit
classification,,
Extreme Greed,0,1
Fear,0,1
Greed,1,2
Neutral,0,1


In [ ]:
# average PnL by sentiment

merged_data.groupby("classification")["total_pnl"].agg(
    ["count", "mean", "median"]
)


,count,mean,median
classification,,,
Extreme Greed,1,1.769655e+05,1.769655e+05
Fear,1,6.699925e+06,6.699925e+06
Greed,3,1.063206e+06,1.555034e+02
Neutral,1,1.587424e+05,1.587424e+05
